In [1]:
from main import TherapyGraph
import json

Connection successful!
Connected to Neo4j database successfully!


In [ ]:
#TherapyGraph.delete_emotions_classes()
#TherapyGraph.insert_emotions_as_nodes()
#TherapyGraph.insert_relationships()

8

In [3]:
TherapyGraph.delete_all_usernames()
TherapyGraph.delete_all_messages()
TherapyGraph.delete_all_emotion_connections()
TherapyGraph.delete_all_people()

TherapyGraph.add_username(name="John Doe", username="user_1")
TherapyGraph.add_username(name="Our_Therapist", username="therapist_1")

All username nodes deleted successfully!
Remaining username nodes: 0
All message nodes deleted successfully!
Remaining message nodes: 0
All emotion connections deleted successfully!
Remaining emotion connections: 0
All person nodes deleted successfully!
Remaining person nodes: 0


<Record u=<Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:3' labels=frozenset({'Username'}) properties={'name': 'Our_Therapist', 'created_at': neo4j.time.DateTime(2025, 5, 27, 4, 23, 10, 831000000, tzinfo=<UTC>), 'username': 'therapist_1'}>>

In [4]:
emotions = TherapyGraph.get_all_emotions(True)


empathy
tenderness
warmth
hostility
resentment
bitterness
forgiveness
acceptance
rejection
approval
disapproval
trust
distrust
confidence
doubt
certainty
uncertainty
optimism
pessimism
serenity
bliss
euphoria
melancholy
loneliness
nostalgia
regret
remorse
compassion
contempt
disdain
awe
wonder
amazement
enthusiasm
passion
desire
lust
affection
joy
happiness
sadness
anger
fear
disgust
surprise
love
hate
excitement
admiration
respect
sympathy
terror
panic
worry
stress
calm
peace
envy
frustration
grief
boredom
gratitude
anxiety
guilt
depression
curiosity
confusion
despair
hope
irritation
fury
relief
shame
rage
jealousy
pride
contentment


In [5]:
from langchain_openai import ChatOpenAI


In [6]:
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0.3)

In [7]:
emotion_names = [emotion['name'] for emotion in emotions]
print(emotion_names)

['empathy', 'tenderness', 'warmth', 'hostility', 'resentment', 'bitterness', 'forgiveness', 'acceptance', 'rejection', 'approval', 'disapproval', 'trust', 'distrust', 'confidence', 'doubt', 'certainty', 'uncertainty', 'optimism', 'pessimism', 'serenity', 'bliss', 'euphoria', 'melancholy', 'loneliness', 'nostalgia', 'regret', 'remorse', 'compassion', 'contempt', 'disdain', 'awe', 'wonder', 'amazement', 'enthusiasm', 'passion', 'desire', 'lust', 'affection', 'joy', 'happiness', 'sadness', 'anger', 'fear', 'disgust', 'surprise', 'love', 'hate', 'excitement', 'admiration', 'respect', 'sympathy', 'terror', 'panic', 'worry', 'stress', 'calm', 'peace', 'envy', 'frustration', 'grief', 'boredom', 'gratitude', 'anxiety', 'guilt', 'depression', 'curiosity', 'confusion', 'despair', 'hope', 'irritation', 'fury', 'relief', 'shame', 'rage', 'jealousy', 'pride', 'contentment']


In [8]:
relationships_labels =  TherapyGraph.get_all_relationships()

In [9]:
relationships = [relatioship["type"] for relatioship in relationships_labels]

In [10]:
def connect_emotions(analysis, i):
     # Create emotion relationships
    if 'emotions' in analysis:
        for emotion_name in analysis['emotions']:
            if emotion_name in emotion_names:
                emotion_node_id = f"emotion_{emotion_name.lower().replace(' ', '_')}"
                print(f"Connecting emotion '{emotion_name}' to message {i}")
                TherapyGraph.connect_emotion(message_id=f"msg_{i}", emotion_name = emotion_name)
            else:
                print(f"Emotion '{emotion_name}' not found in the list.")
                continue

In [ ]:
from pydantic import BaseModel
from typing import List, Optional

with open('test_msg.json', 'r') as file:
    agent_message = json.load(file)


for message in agent_message["conversation"]:
    print(f"Adding message {message['message_id']} from {message['role']}")
    msg = message["message"]
    print(f"Message content: {msg}")
    username = "therapist_1" if message["role"] == "agent" else "user_1"
    TherapyGraph.add_message_node(text=msg, conversation_id="conv_1", message_id=message["message_id"], username=username)
    
    # Use LLM to analyze the message for emotions and relationships
    class EmotionRelationshipAnalysis(BaseModel):
        emotions: Optional[List[str]] = None
        relationships: Optional[List[str]] = None

    try: 
        prompt = f"""Analyze this message to identify:
        1. Emotions expressed or implied in the message
        2. People or relationships the user is talking about
        
        Return a JSON object with:
        - 'emotions' field: list of emotions from this list {emotion_names} or null if none found
        - 'relationships' field: list of people/relationship from this list {relationships} or null if none found
        
        Message: '{msg}'
        
        Expected format: {{"emotions": ["emotion1", "emotion2"], "relationships": ["person1", "person2"]}}
        Use null for fields where nothing is found."""
        
        response = llm.invoke(prompt)
        
        # Parse and validate the response using Pydantic
        analysis_data = json.loads(response.content)
        analysis = EmotionRelationshipAnalysis(**analysis_data)
        
        print(f"Analysis for message {message['message_id']}: {analysis.model_dump()}")
        
        # Connect emotions
        if analysis.emotions:
            connect_emotions(analysis.model_dump(), message['message_id'])
        
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        analysis = EmotionRelationshipAnalysis(emotions=None, relationships=None)
    except Exception as e:
        print(f"Error analyzing message: {e}")
        analysis = EmotionRelationshipAnalysis(emotions=None, relationships=None)

    if analysis.relationships is not None:
        for person in analysis.relationships:
            print(f"Creating relationship node for: {person}")
            TherapyGraph.connect_person(name=person, username=username, role="relationship")
    else:
        print("No relationships found in this message.")


    

Adding message msg_001 from agent
Message content: hello, how can i help you today? 
Analysis for message msg_001: {'emotions': ['helpfulness'], 'relationships': None}
Emotion 'helpfulness' not found in the list.
No relationships found in this message.
Adding message msg_002 from user
Message content: hi, my dad beat me when i was child i think of this sometimes like now, that makes me sad. 
Analysis for message msg_002: {'emotions': ['sadness'], 'relationships': ['dad']}
Connecting emotion 'sadness' to message msg_002
Emotion 'sadness' added to message 'msg_msg_002' successfully!
Creating relationship node for: dad
Adding message msg_003 from agent
Message content: that is not very good, how does that affect you now? 
Analysis for message msg_003: {'emotions': ['concern', 'worry'], 'relationships': None}
Emotion 'concern' not found in the list.
Connecting emotion 'worry' to message msg_003
Emotion 'worry' added to message 'msg_msg_003' successfully!
No relationships found in this mess